In [4]:
import json
from unskript import nbparams
from unskript.secrets import ENV_MODE, ENV_MODE_AWS
from unskript.fwk.workflow import Task, Workflow

env = {"ENV_MODE":"ENV_MODE_AWS","TENANT_ID":"982dba5f-d9df-48ae-a5bf-ec1fc94d4882","ENVIRONMENT_ID":"1499f27c-6406-4fbd-bd1b-c6f92800018f","TENANT_URL":"https://tenant-staging.alpha.unskript.io","AWS_REGION":"us-west-2"}
secret_store_cfg = {"SECRET_STORE_TYPE":"SECRET_STORE_TYPE_AWS","AWS_SECRET_PREFIX":"staging","AWS_REGION":"us-west-2"}
paramDict = {}
paramDict.update(env)
paramDict.update(secret_store_cfg)
paramsJson = json.dumps(paramDict)
nbParamsObj = nbparams.NBParams(paramsJson)

w = Workflow(env, secret_store_cfg, None, global_vars=globals())

<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates how to check SSL Certificate expiration date and send a reminder message to the Slack channel using unSkript legos.</b>
</div>

<br>

<center><h2>Renew SSL Certificate</h2></center>

# Steps Overview
    1) Fetch SSL Certificate expiration date from the AWS ACM (Amazon Certificate Manager).
    2) Send a message to Slack ONLY if the number of days remaining are less than 30 days.

In this lego we will check the expiration date of the SSL Certificate and return the number of days remaining before renewal

In [5]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
import pprint
import datetime
import dateutil


from beartype import beartype
@beartype
def aws_check_ssl_certificate_expiry_printer(output):
    if output is None:
        return
    pprint.pprint("Your SSL certificate is expiring in " + str(output) + " " + "days")


@beartype
def aws_check_ssl_certificate_expiry(
    handle,
    aws_certificate_arn: str,
    region: str,
) -> int:
    iamClient = handle.client('acm', region_name=region)
    result = iamClient.describe_certificate(CertificateArn=aws_certificate_arn)
    for k,v in result['Certificate'].items():
        if k == "NotAfter":
            val = v
            right_now = datetime.datetime.now(dateutil.tz.tzlocal())
            diff = val-right_now
            days_remaining = diff.days
            if days_remaining < 30:
                return days_remaining


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "DevRole",
    "credential_type": "CONNECTOR_TYPE_AWS",
    "credential_id": "0b438eba-0627-4f6d-b998-a4c604f20e3c"
}''')
task.configure(inputParamsJson='''{
    "aws_certificate_arn": "\\"arn:aws:acm:us-west-2:100498623390:certificate/22861cb3-313e-44b9-b0c4-1a10a6f6f105\\"",
    "region": "\\"us-west-2\\""
    }''')
task.configure(outputName="days_remaining")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_check_ssl_certificate_expiry, lego_printer=aws_check_ssl_certificate_expiry_printer, hdl=hdl, args=args)

Here we will take the number of days remaining as input and check in the 'Start Condition' if the days remaining are less than 30 and if yes, send a message on the selected slack channel 

In [6]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import pprint

from pydantic import BaseModel, Field
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

pp = pprint.PrettyPrinter(indent=2)


from beartype import beartype
@beartype
def slack_post_message_printer(output):
    if output is not None:
        pprint.pprint(output)
    else:
        return


@beartype
def slack_post_message(
        handle: WebClient,
        channel: str,
        message: str) -> str:

    try:
        response = handle.chat_postMessage(
            channel=channel,
            text=message)
        return f"Successfuly Sent Message on Channel: #{channel}"
    except SlackApiError as e:
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.response['error']}")
        if e.response['error'] == 'channel_not_found':
            raise Exception('Channel Not Found')
        elif e.response['error'] == 'duplicate_channel_not_found':
            raise Exception('Channel associated with the message_id not valid')
        elif e.response['error'] == 'not_in_channel':
            raise Exception('Cannot post message to channel user is not in')
        elif e.response['error'] == 'is_archived':
            raise Exception('Channel has been archived')
        elif e.response['error'] == 'msg_too_long':
            raise Exception('Message text is too long')
        elif e.response['error'] == 'no_text':
            raise Exception('Message text was not provided')
        elif e.response['error'] == 'restricted_action':
            raise Exception('Workspace preference prevents user from posting')
        elif e.response['error'] == 'restricted_action_read_only_channel':
            raise Exception('Cannot Post message, read-only channel')
        elif e.response['error'] == 'team_access_not_granted':
            raise Exception('The token used is not granted access to the workspace')
        elif e.response['error'] == 'not_authed':
            raise Exception('No Authtnecition token provided')
        elif e.response['error'] == 'invalid_auth':
            raise Exception('Some aspect of Authentication cannot be validated. Request denied')
        elif e.response['error'] == 'access_denied':
            raise Exception('Access to a resource specified in the request denied')
        elif e.response['error'] == 'account_inactive':
            raise Exception('Authentication token is for a deleted user')
        elif e.response['error'] == 'token_revoked':
            raise Exception('Authentication token for a deleted user has been revoked')
        elif e.response['error'] == 'no_permission':
            raise Exception('The workspace toekn used does not have necessary permission to send message')
        elif e.response['error'] == 'ratelimited':
            raise Exception('The request has been ratelimited. Retry sending message later')
        elif e.response['error'] == 'service_unavailable':
            raise Exception('The service is temporarily unavailable')
        elif e.response['error'] == 'fatal_error':
            raise Exception('The server encountered catostrophic error while sending message')
        elif e.response['error'] == 'internal_error':
            raise Exception('The server could not complete operation, likely due to transietn issue')
        elif e.response['error'] == 'request_timeout':
            raise Exception('Sending message error via POST: either message was missing or truncated')
        else:
            raise Exception(f'Failed Sending Message to slack channel {channel} Error: {e.response["error"]}')

    except Exception as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.__str__()}")
        return f"Unable to send message on {channel}"


task = Task(Workflow())
task.configure(credentialsJson='''{
    "credential_name": "unSkript",
    "credential_type": "CONNECTOR_TYPE_SLACK",
    "credential_id": "b1adafe4-6593-484d-8fc1-e40799378e9a"
}''')
task.configure(inputParamsJson='''{
    "channel": "\\"dummy\\"",
    "message": "\\"Your SSL certificate is expiring in {} days\\".format(days_remaining)"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "days_remaining<30",
    "condition_result": true
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(slack_post_message, lego_printer=slack_post_message_printer, hdl=hdl, args=args)